# I saved pre processed text in csv file m using that dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk import WordNetLemmatizer
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.optimizers import Adam

In [2]:
urdu_dataset = pd.read_csv("Analysed_Urdu_Tweets.csv",index_col=False)
urdu_dataset.drop(urdu_dataset.columns[0],axis=1,inplace=True)
urdu_dataset.head()


,Tweet,Class
0,ایٹم بم بنایا ھے بھائی ایٹم بمب کوٹ لکھپت اتفا...,2
1,چندے انقلاب عمران خان وزیر اعظم بن سکتے,0
2,ٹویٹر خیال,1
3,سرچ انجن گوگل نائب صدر فضا فٹ بلندی چھلانگ عال...,2
4,اسکی لہریں یار أ,2


In [3]:

## WORDS in dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(urdu_dataset['Tweet'])
sequences = tokenizer.texts_to_sequences(urdu_dataset['Tweet'])

max_sequence_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
## Split the dataset
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, urdu_dataset['Class'], test_size=0.25, random_state=42)

print("Training set shape:",X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (749, 31)
Testing set shape: (250, 31)


# Models

In [18]:
import gensim.downloader as api
import tensorflow_hub as hub
word2vec_model = api.load("word2vec-google-news-300")
# glove_model = api.load("glove-twitter-200")
glove_vectors = api.load("glove-wiki-gigaword-100")
fasttext_model = api.load("fasttext-wiki-news-subwords-300")
elmo = hub.load("https://tfhub.dev/google/elmo/2")

KeyboardInterrupt: 

In [19]:
glove_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
max_length = max([len(seq) for seq in X_train])
X_train_padded = pad_sequences(X_train, maxlen=max_length)
X_test_padded = pad_sequences(X_test, maxlen=max_length)

In [6]:
# word2vec_weights = create_embedding_matrix(word2vec_model, tokenizer)

vocab_size = len(tokenizer.word_index) + 1
embed_matrix = np.zeros((vocab_size,word2vec_model.vector_size))

for w,i in tokenizer.word_index.items():
    if w in word2vec_model:
        embed_matrix[i] = word2vec_model[w]
word2vec_weights = embed_matrix

In [7]:
# LSTM -> 3,0.7 ->0.66
# nested for for dropout and num_layers

num_layers =3
dropout_rate=0.7

embedding_dim = 128

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(units=64))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=32, epochs=8, validation_split=0.25)

Epoch 1/8
18/18 [==============================] - 13s 174ms/step - loss: 0.0000e+00 - accuracy: 0.4082 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 2/8
18/18 [==============================] - 1s 70ms/step - loss: 0.0000e+00 - accuracy: 0.4545 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 3/8
18/18 [==============================] - 1s 72ms/step - loss: 0.0000e+00 - accuracy: 0.4100 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 4/8
18/18 [==============================] - 1s 74ms/step - loss: 0.0000e+00 - accuracy: 0.4545 - val_loss: 0.0000e+00 - val_accuracy: 0.2234
Epoch 5/8
18/18 [==============================] - 2s 84ms/step - loss: 0.0000e+00 - accuracy: 0.4670 - val_loss: 0.0000e+00 - val_accuracy: 0.1968
Epoch 6/8
18/18 [==============================] - 1s 77ms/step - loss: 0.0000e+00 - accuracy: 0.4920 - val_loss: 0.0000e+00 - val_accuracy: 0.2819
Epoch 7/8
18/18 [==============================] - 2s 103ms/step - loss: 0.0000e+00 - accuracy: 0.4955 - val_l

In [9]:
model_eval = dict()
from sklearn.metrics import precision_score, recall_score, f1_score

In [10]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
loss, accuracy = model.evaluate(X_test, y_test)

model_eval['LSTM(no embedding)'] = {'Precision':precision,'Recall':recall,'F1 score':f1,'accuracy':accuracy}

8/8 [==============================] - 0s 20ms/step - loss: 0.0000e+00 - accuracy: 0.2880


In [11]:
model_eval

{'LSTM(no embedding)': {'Precision': 0.21623376623376625,
  'Recall': 0.3551912568306011,
  'F1 score': 0.2119744922059518,
  'accuracy': 0.2879999876022339}}

## word2vec

In [12]:
model = Sequential()
model.add(Embedding(input_dim=len(word2vec_weights), output_dim=300,weights=[word2vec_weights], input_length=max_length))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(units=64))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
10/10 [==============================] - 11s 381ms/step - loss: 0.0000e+00 - accuracy: 0.2371 - val_loss: 0.0000e+00 - val_accuracy: 0.0067
Epoch 2/10
10/10 [==============================] - 1s 131ms/step - loss: 0.0000e+00 - accuracy: 0.2721 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 3/10
10/10 [==============================] - 1s 129ms/step - loss: 0.0000e+00 - accuracy: 0.3656 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 4/10
10/10 [==============================] - 1s 138ms/step - loss: 0.0000e+00 - accuracy: 0.4808 - val_loss: 0.0000e+00 - val_accuracy: 0.4933
Epoch 5/10
10/10 [==============================] - 2s 155ms/step - loss: 0.0000e+00 - accuracy: 0.4841 - val_loss: 0.0000e+00 - val_accuracy: 0.3200
Epoch 6/10
10/10 [==============================] - 1s 143ms/step - loss: 0.0000e+00 - accuracy: 0.5058 - val_loss: 0.0000e+00 - val_accuracy: 0.3067
Epoch 7/10
10/10 [==============================] - 1s 146ms/step - loss: 0.0000e+00 - accuracy: 0.

In [14]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision2 = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall2 = recall_score(y_test, y_pred, average='macro', zero_division=0)
f12 = f1_score(y_test, y_pred, average='macro', zero_division=0)
loss, accuracy2 = model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 25ms/step - loss: 0.0000e+00 - accuracy: 0.3800


In [15]:
model_eval['LSTM(embedding: word2vec)'] = {'Precision':precision2,'Recall':recall2,'F1 score':f12,'accuracy':accuracy2}

In [16]:
table = pd.DataFrame.from_dict(model_eval, orient='index')
print(table)

                           Precision    Recall  F1 score  accuracy
LSTM(no embedding)          0.216234  0.355191  0.211974     0.288
LSTM(embedding: word2vec)   0.190573  0.312386  0.223668     0.380


## glove

embed matrix

In [20]:
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in glove_vectors.key_to_index:
        embedding_matrix[i] = glove_vectors[word]

model 

In [21]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(units=3, activation='softmax')) #3 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
12/12 [==============================] - 7s 148ms/step - loss: 1.0097 - accuracy: 0.4980 - val_loss: 0.8262 - val_accuracy: 0.4880
Epoch 2/10
12/12 [==============================] - 1s 70ms/step - loss: 0.7971 - accuracy: 0.4900 - val_loss: 0.7908 - val_accuracy: 0.4880
Epoch 3/10
12/12 [==============================] - 1s 73ms/step - loss: 0.7820 - accuracy: 0.4806 - val_loss: 0.7891 - val_accuracy: 0.4880
Epoch 4/10
12/12 [==============================] - 1s 68ms/step - loss: 0.7754 - accuracy: 0.5033 - val_loss: 0.7937 - val_accuracy: 0.4880
Epoch 5/10
12/12 [==============================] - 1s 73ms/step - loss: 0.7798 - accuracy: 0.5060 - val_loss: 0.8074 - val_accuracy: 0.4880
Epoch 6/10
12/12 [==============================] - 1s 85ms/step - loss: 0.7901 - accuracy: 0.4806 - val_loss: 0.7892 - val_accuracy: 0.4880
Epoch 7/10
12/12 [==============================] - 1s 83ms/step - loss: 0.7731 - accuracy: 0.5007 - val_loss: 0.7945 - val_accuracy: 0.4880
Epoch 8/10
1

In [23]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

8/8 [==============================] - 1s 19ms/step


Evaluate

In [24]:
loss, accuracy3 = model.evaluate(X_test, y_test)
precision3 = precision_score(y_test, y_pred_labels, average='macro')
recall3 = recall_score(y_test, y_pred_labels, average='macro')
f13 = f1_score(y_test, y_pred_labels, average='macro')
model_eval['LSTM(embedding: GLOVE)'] = {'Precision':precision3,'Recall':recall3,'F1 score':f13,'accuracy':accuracy3}

1/8 [==>...........................] - ETA: 0s - loss: 0.7088 - accuracy: 0.5625

8/8 [==============================] - 0s 27ms/step - loss: 0.7918 - accuracy: 0.4960


d:\Python64\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Fast Text

In [25]:
fasttext_model = api.load("fasttext-wiki-news-subwords-300")

Embed matrix

In [28]:
embed_matrix = np.zeros((vocab_size,fasttext_model.vector_size))

for w,i in tokenizer.word_index.items():
    if w in fasttext_model and w in word2vec_model:
        embed_matrix[i] = word2vec_model[w]
fasttext_weights = embed_matrix    


Model

In [29]:
input_dim = len(fasttext_weights)

model=Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, weights=[embedding_matrix], input_length=input_length, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


NameError: name 'output_dim' is not defined

In [ ]:
model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.2)


In [ ]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
loss, accuracy4 = model.evaluate(X_test, y_test)
model_eval['LSTM(embedding: fast-text)'] = {'Precision':precision,'Recall':recall,'F1 score':f1,'accuracy':accuracy4}

Elmo

In [ ]:
elmo = hub.load("https://tfhub.dev/google/elmo/2")

In [ ]:
table = pd.DataFrame.from_dict(model_eval, orient='index')
print(table)

                          Precision    Recall  F1 score  accuracy
LSTM(no embedding)         0.162667  0.333333  0.218638       NaN
GRU(embedding: word2vec)   0.162667  0.333333  0.218638       NaN
LSTM(embedding: GLOVE)     0.162667  0.333333  0.218638     0.488
